In [13]:
import ollama
import numpy as np
import requests
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class OllamaEmbeddingSearch:
    def __init__(self, model="nomic-embed-text"):
        self.model = model
        self.data_store = {}

    def add_data(self, identifier, text):
        """Generates an embedding for the given text and stores it."""
        logger.info(f"Generating embedding for: {identifier}")
        try:
            embedding = np.array(ollama.embed(model=self.model, input=text)["embeddings"]).squeeze()
            self.data_store[identifier] = embedding
        except Exception as e:
            logger.error(f"Error generating embedding: {e}")

    def search(self, query, top_k=1):
        """Finds the most similar text in the stored data based on cosine similarity."""
        logger.info(f"Searching for: {query}")
        try:
            query_embedding = np.array(ollama.embed(model=self.model, input=query)["embeddings"]).squeeze()

            similarities = {}
            for identifier, stored_embedding in self.data_store.items():
                similarity = np.dot(query_embedding, stored_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(stored_embedding))
                similarities[identifier] = similarity
            
            sorted_results = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
            return sorted_results[:top_k]
        except Exception as e:
            logger.error(f"Error in search: {e}")
            return []

# Example Usage
embedding_search = OllamaEmbeddingSearch()
embedding_search.add_data("doc1", "Ollama is a powerful AI tool.")
embedding_search.add_data("doc2", "Machine learning models can be used for text processing.")

query = "Tell me about AI tools."
logger.info(embedding_search.search(query))


INFO:__main__:Generating embedding for: doc1
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:__main__:Generating embedding for: doc2
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:__main__:Searching for: Tell me about AI tools.
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:__main__:[('doc1', np.float64(0.6860680795567213))]


In [ ]:
import json
import ollama

# Simulates an API call to get flight times
# In a real application, this would fetch data from a live database or API
def get_flight_times(departure: str, arrival: str) -> str:
    flights = {
        "NYC-LAX": {
            "departure": "08:00 AM",
            "arrival": "11:30 AM",
            "duration": "5h 30m",
        },
        "LAX-NYC": {
            "departure": "02:00 PM",
            "arrival": "10:30 PM",
            "duration": "5h 30m",
        },
        "LHR-JFK": {
            "departure": "10:00 AM",
            "arrival": "01:00 PM",
            "duration": "8h 00m",
        },
        "JFK-LHR": {
            "departure": "09:00 PM",
            "arrival": "09:00 AM",
            "duration": "7h 00m",
        },
        "CDG-DXB": {
            "departure": "11:00 AM",
            "arrival": "08:00 PM",
            "duration": "6h 00m",
        },
        "DXB-CDG": {
            "departure": "03:00 AM",
            "arrival": "07:30 AM",
            "duration": "7h 30m",
        },
    }

    key = f"{departure}-{arrival}".upper()
    return json.dumps(flights.get(key, {"error": "Flight not found"}))



# Search data related to Artificial Intelligence in a vector database
def search_data_in_vector_db(query: str) -> str:
    query_vectors = embedding_fn.encode_queries([query])

    res = client.search(
        collection_name="demo_collection",
        data=query_vectors,
        limit=2,
        output_fields=["text", "subject"],  # specifies fields to be returned
    )
    print(res)
    return json.dumps(res)

In [2]:
def run(model: str, question: str):
    client = ollama.Client()
    # Initialize conversation with a user query
    messages = [{"role": "user", "content": question}]

    # First API call: Send the query and function description to the model
    response = client.chat(
        model=model,
        messages=messages,
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "get_flight_times",
                    "description": "Get the flight times between two cities",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "departure": {
                                "type": "string",
                                "description": "The departure city (airport code)",
                            },
                            "arrival": {
                                "type": "string",
                                "description": "The arrival city (airport code)",
                            },
                        },
                        "required": ["departure", "arrival"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "search_data_in_vector_db",
                    "description": "Search about Artificial Intelligence data in a vector database",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "The search query",
                            },
                        },
                        "required": ["query"],
                    },
                },
            },
        ],
    )

    # Add the model's response to the conversation history
    messages.append(response["message"])

    # Check if the model decided to use the provided function
    if not response["message"].get("tool_calls"):
        print("The model didn't use the function. Its response was:")
        print(response["message"]["content"])
        return

    # Process function calls made by the model
    if response["message"].get("tool_calls"):
        available_functions = {
            "get_flight_times": get_flight_times,
            "search_data_in_vector_db": search_data_in_vector_db,
        }
        for tool in response["message"]["tool_calls"]:
            function_to_call = available_functions[tool["function"]["name"]]
            function_args = tool["function"]["arguments"]
            function_response = function_to_call(**function_args)
            # Add function response to the conversation
            messages.append(
                {
                    "role": "tool",
                    "content": function_response,
                }
            )

    # Second API call: Get final response from the model
    final_response = client.chat(model=model, messages=messages)
    print(final_response["message"]["content"])

In [3]:
question = "What is the flight time from New York (NYC) to Los Angeles (LAX)?"
run("llama3.2", question)

The flight time from New York (NYC) to Los Angeles (LAX) can vary depending on several factors such as the airline, route, and weather conditions. However, on average, a direct flight from New York to Los Angeles takes around 5 hours and 30 minutes to 6 hours.

It's worth noting that there are no direct flights from New York to Los Angeles. Most flights have a layover or connection in another city before reaching their destination. The total travel time, including check-in, security screening, boarding, and any connections, can range from 8 to 12 hours or more, depending on the specific flight schedule and airline.

To get a more accurate estimate of flight times, I recommend checking with airlines or flight search engines such as Google Flights, Skyscanner, or Kayak.


In [4]:
question = "When was Artificial Intelligence founded?"
run("llama3.2", question)

NameError: name 'embedding_fn' is not defined

In [ ]:
import ollama

def add_numbers(a, b):
    return a + b

response = ollama.generate("Add 5 and 7.", functions=[add_numbers])
print(response['output'])